In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

C:\Users\lfdin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Informações sobre o dataset

Esse dataset foi construído a partir de informações em anúncios de venda de veículos disponíveis online, entre os dias 30/06/2022 e 04/07/2022. Na obtenção dos dados, foram procurados apenas anúncios profissionais de veículos com ano de fabricação a partir de 2000. Além disso, foi verificado no portal de vendas uma quantidade aproximadamente igual de anúncios entre as seguintes faixas de quilometragem: 0-30, 30-60, 60-100, 100+. Sendo assim, de forma a tornar a amostragem realizada mais representativa, os valores de quilometragem nesse dataset seguem uma proporção parecida.

In [2]:
df = pd.read_csv('cars_data_final.csv')

df.head()

,Marca,Descricao,Tipo,Ano,UF,Km,Potencia,Combustivel,Cambio,Direcao,...,CameraDeRe,Blindado,Preco,Modelo,Dias,KmLimite,KmPorDias,NumOpcionais,KmSqrt,PotenciaNum
0,JEEP,JEEP COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,SUV,2017,DF,0,2.0 - 2.9,Diesel,Automático,Elétrica,...,True,False,139800.0,COMPASS,2194,767900,0.000000,7,0.000000,2.5
1,VW - VOLKSWAGEN,VW - VOLKSWAGEN SAVEIRO 1.6 MI/ 1.6 MI TOTAL F...,Pick-up,2014,RS,1,1.6,Flex,Manual,Hidráulica,...,False,False,49900.0,SAVEIRO,3290,1151500,0.000304,6,1.000000,1.6
2,TOYOTA,TOYOTA ETIOS,Sedã,2015,PA,1000,1.5,Flex,Manual,Hidráulica,...,False,False,47900.0,ETIOS,2925,1023750,0.341880,6,31.622777,1.5
3,VW - VOLKSWAGEN,VW - VOLKSWAGEN POLO COMFORT. 200 TSI 1.0 FLEX...,Hatch,2021,PR,11139,1.0,Flex,Automático,Hidráulica,...,False,False,98900.0,POLO,733,256550,15.196453,6,105.541461,1.0
4,HYUNDAI,HYUNDAI HB20S VISION 1.6 FLEX 16V AUT,Sedã,2021,CE,25223,1.6,Flex,Automático,Hidráulica,...,True,False,80899.0,HB20S,733,256550,34.410641,8,158.817505,1.6


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5167 entries, 0 to 5166
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Marca           5167 non-null   object 
 1   Descricao       5167 non-null   object 
 2   Tipo            5167 non-null   object 
 3   Ano             5167 non-null   int64  
 4   UF              5167 non-null   object 
 5   Km              5167 non-null   int64  
 6   Potencia        5167 non-null   object 
 7   Combustivel     5167 non-null   object 
 8   Cambio          5167 non-null   object 
 9   Direcao         5167 non-null   object 
 10  Cor             5167 non-null   object 
 11  FinalDePlaca    5167 non-null   int64  
 12  Portas          5167 non-null   object 
 13  UnicoDono       5167 non-null   object 
 14  AceitaTrocas    5167 non-null   object 
 15  ArCondicionado  5167 non-null   bool   
 16  AirBag          5167 non-null   bool   
 17  Alarme          5167 non-null   b

# Estabelecendo o benchmark: Tabela Fipe

O objetivo desse projeto é desenvolver um modelo capaz de oferecer estimativas de preços mais realistas do que aquelas fornecidas pela Tabela Fipe, baseadas no que está sendo de fato praticado no mercado e levando em consideração mais variáveis do que a tabela de consulta padrão para compra e venda de automóveis.

In [5]:
df_fipe = pd.read_csv('cars_data_fipe.csv')

df_fipe.head()

,Marca,Descricao,Tipo,Ano,UF,Km,Potencia,Combustivel,Cambio,Direcao,...,PotenciaNum,CodMarca,DescricaoLimpa,DescricaoFipe,CodModelo,AnoCombustivel,AnoFipe,CodAno,ChaveFipe,PrecoFipe
0,JEEP,JEEP COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,SUV,2017,DF,0,2.0 - 2.9,Diesel,Automático,Elétrica,...,2.5,29,COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,7760,2017 Diesel,2017 Diesel,2017-3,29 7760 2017-3,139372.0
1,VW - VOLKSWAGEN,VW - VOLKSWAGEN SAVEIRO 1.6 MI/ 1.6 MI TOTAL F...,Pick-up,2014,RS,1,1.6,Flex,Manual,Hidráulica,...,1.6,59,SAVEIRO 1.6 MI/ 1.6 MI TOTAL FLEX 8V,SAVEIRO 1.6 MI/ 1.6 MI TOTAL FLEX 8V,2553,2014 Gasolina,2014 Gasolina,2014-1,59 2553 2014-1,44834.0
2,TOYOTA,TOYOTA ETIOS,Sedã,2015,PA,1000,1.5,Flex,Manual,Hidráulica,...,1.5,56,ETIOS,ETIOS 1.3 FLEX 16V 5P MEC.,6245,2015 Gasolina,2014 Gasolina,2014-1,56 6245 2014-1,45249.0
3,VW - VOLKSWAGEN,VW - VOLKSWAGEN POLO COMFORT. 200 TSI 1.0 FLEX...,Hatch,2021,PR,11139,1.0,Flex,Automático,Hidráulica,...,1.0,59,POLO COMFORT. 200 TSI 1.0 FLEX 12V AUT.,POLO COMFORT. 200 TSI 1.0 FLEX 12V AUT.,8068,2021 Gasolina,2021 Gasolina,2021-1,59 8068 2021-1,91709.0
4,HYUNDAI,HYUNDAI HB20S VISION 1.6 FLEX 16V AUT,Sedã,2021,CE,25223,1.6,Flex,Automático,Hidráulica,...,1.6,26,HB20S VISION 1.6 FLEX 16V AUT,HB20S VISION 1.6 FLEX 16V AUT.,8859,2021 Gasolina,2021 Gasolina,2021-1,26 8859 2021-1,81168.0


Dado que as informações fornecidas sobre o veículo são preenchidas pelos próprios revendedores, algumas distorções são observadas. Em alguns casos, por exemplo, o modelo do carro citado não é compatível com o ano de fabricação do mesmo, o que impacta principalmente a consulta à tabela Fipe. Sendo assim, será criada uma flag que indica se a informação fornecida pode estar distorcida em relação à consulta realizada na Tabela Fipe.

- `DifAnosFipe`: Diferença de anos observada entre o anúncio e a Tabela Fipe;
- `FlagFipe`: Indica se o modelo é compatível com o consultado na tabela Fipe (True) ou não (False). O primeiro pressuposto para isso é que a descrição do modelo deve ser igual àquela presente na Tabela Fipe, salvo algumas diferenças de pontuação. O segundo pressuposto assumido é de que não deve existir diferença entre o ano informado para o modelo e aquele consultado na Tabela Fipe.

In [6]:
df_fipe['DifAnosFipe'] = df_fipe.apply(lambda x: x['Ano'] - int(x['AnoFipe'].split(' ')[0]), axis=1)

df_fipe.head()

,Marca,Descricao,Tipo,Ano,UF,Km,Potencia,Combustivel,Cambio,Direcao,...,CodMarca,DescricaoLimpa,DescricaoFipe,CodModelo,AnoCombustivel,AnoFipe,CodAno,ChaveFipe,PrecoFipe,DifAnosFipe
0,JEEP,JEEP COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,SUV,2017,DF,0,2.0 - 2.9,Diesel,Automático,Elétrica,...,29,COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,7760,2017 Diesel,2017 Diesel,2017-3,29 7760 2017-3,139372.0,0
1,VW - VOLKSWAGEN,VW - VOLKSWAGEN SAVEIRO 1.6 MI/ 1.6 MI TOTAL F...,Pick-up,2014,RS,1,1.6,Flex,Manual,Hidráulica,...,59,SAVEIRO 1.6 MI/ 1.6 MI TOTAL FLEX 8V,SAVEIRO 1.6 MI/ 1.6 MI TOTAL FLEX 8V,2553,2014 Gasolina,2014 Gasolina,2014-1,59 2553 2014-1,44834.0,0
2,TOYOTA,TOYOTA ETIOS,Sedã,2015,PA,1000,1.5,Flex,Manual,Hidráulica,...,56,ETIOS,ETIOS 1.3 FLEX 16V 5P MEC.,6245,2015 Gasolina,2014 Gasolina,2014-1,56 6245 2014-1,45249.0,1
3,VW - VOLKSWAGEN,VW - VOLKSWAGEN POLO COMFORT. 200 TSI 1.0 FLEX...,Hatch,2021,PR,11139,1.0,Flex,Automático,Hidráulica,...,59,POLO COMFORT. 200 TSI 1.0 FLEX 12V AUT.,POLO COMFORT. 200 TSI 1.0 FLEX 12V AUT.,8068,2021 Gasolina,2021 Gasolina,2021-1,59 8068 2021-1,91709.0,0
4,HYUNDAI,HYUNDAI HB20S VISION 1.6 FLEX 16V AUT,Sedã,2021,CE,25223,1.6,Flex,Automático,Hidráulica,...,26,HB20S VISION 1.6 FLEX 16V AUT,HB20S VISION 1.6 FLEX 16V AUT.,8859,2021 Gasolina,2021 Gasolina,2021-1,26 8859 2021-1,81168.0,0


In [8]:
df_fipe['FlagFipe'] = df_fipe.apply(lambda x: True if ((abs(x['DifAnosFipe'])==0)and(x['DescricaoLimpa'].replace('.', '')==x['DescricaoFipe'].replace('.', ''))) else False, axis=1)

df.head()

,Marca,Descricao,Tipo,Ano,UF,Km,Potencia,Combustivel,Cambio,Direcao,...,CameraDeRe,Blindado,Preco,Modelo,Dias,KmLimite,KmPorDias,NumOpcionais,KmSqrt,PotenciaNum
0,JEEP,JEEP COMPASS LONGITUDE 2.0 4X4 DIES. 16V AUT.,SUV,2017,DF,0,2.0 - 2.9,Diesel,Automático,Elétrica,...,True,False,139800.0,COMPASS,2194,767900,0.000000,7,0.000000,2.5
1,VW - VOLKSWAGEN,VW - VOLKSWAGEN SAVEIRO 1.6 MI/ 1.6 MI TOTAL F...,Pick-up,2014,RS,1,1.6,Flex,Manual,Hidráulica,...,False,False,49900.0,SAVEIRO,3290,1151500,0.000304,6,1.000000,1.6
2,TOYOTA,TOYOTA ETIOS,Sedã,2015,PA,1000,1.5,Flex,Manual,Hidráulica,...,False,False,47900.0,ETIOS,2925,1023750,0.341880,6,31.622777,1.5
3,VW - VOLKSWAGEN,VW - VOLKSWAGEN POLO COMFORT. 200 TSI 1.0 FLEX...,Hatch,2021,PR,11139,1.0,Flex,Automático,Hidráulica,...,False,False,98900.0,POLO,733,256550,15.196453,6,105.541461,1.0
4,HYUNDAI,HYUNDAI HB20S VISION 1.6 FLEX 16V AUT,Sedã,2021,CE,25223,1.6,Flex,Automático,Hidráulica,...,True,False,80899.0,HB20S,733,256550,34.410641,8,158.817505,1.6


In [9]:
len(df_fipe.loc[df_fipe['FlagFipe']==False])

1508

Temos 1508 observações com potenciais distorções em relação à tabela Fipe. Como a mesma servirá de baseline da performance do modelo desenvolvido, esses registros serão eliminados.

In [10]:
df.drop(df_fipe.loc[df_fipe['FlagFipe']==False].index, inplace=True)
df_fipe.drop(df_fipe.loc[df_fipe['FlagFipe']==False].index, inplace=True)

len(df_fipe), len(df)

(3659, 3659)

# Dividindo a base de dados

Como primeiro passo, a variável alvo será separada da base de dados, que por sua vez será dividida entre amostras de treino (aproximadamente 70% das observações), validação (15% das observações) e teste (15% das observações). A feature `Descricao` será eliminada do dataset por não ser muito informativa.

In [12]:
from sklearn.model_selection import train_test_split

y = df['Preco']

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Preco','Descricao'], axis=1), y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

len(X_train), len(X_valid), len(X_test)

(2561, 549, 549)

A base com valores da tabela Fipe também deve ser dividida:

In [13]:
y_fipe = df_fipe['PrecoFipe']

X_fipe_train, X_fipe_test, y_fipe_train, y_fipe_test = train_test_split(df_fipe.drop(['Preco','PrecoFipe','Descricao','DescricaoLimpa','DescricaoFipe','CodMarca','CodModelo','AnoCombustivel','AnoFipe','CodAno','ChaveFipe','FlagFipe'], axis=1), y_fipe, test_size=0.3, random_state=42)
X_fipe_valid, X_fipe_test, y_fipe_valid, y_fipe_test = train_test_split(X_fipe_test, y_fipe_test, test_size=0.5, random_state=42)

len(X_fipe_train), len(X_fipe_valid), len(X_fipe_test)

(2561, 549, 549)

Dado que o hiperparâmetro `random_state` foi mantido o mesmo, as bases de treino, validação e teste dos dois dataframes possuem exatamente as mesmas observações.

As métricas de avaliação consideradas nesse projeto serão três: RMSE, MAPE e RMSLE.

- **RMSE**: essa é uma das métricas mais populares, que penaliza os erros maiores. Tem como grande vantagem reportar um erro na unidade da variável dependente, sendo esse de fácil interpretação. Ou seja, para o caso avaliado, a métrica RMSE apresentará um erro em termos de reais (R\$);


- **MAPE**: essa métrica calcula o erro em termos de uma porcentagem em relação ao valor real. Ela também é fácil de se interpretar e não depende da escala dos valores;


- **RMSLE**: essa métrica aplica o log aos valores real e predito antes de registrar suas diferenças. Isso a torna robusta a outliers. No entanto, a subestimação é mais penalizada que a sobrestimação dos valores.

In [14]:
df_metricas = pd.DataFrame(columns=['Modelo', 'RMSE', 'MAPE', 'RMSLE'])

df_metricas

,Modelo,RMSE,MAPE,RMSLE


In [15]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error

rmse  = mean_squared_error(y_valid, y_fipe_valid, squared=False)
mape  = mean_absolute_percentage_error(y_valid, y_fipe_valid)
rmsle = mean_squared_log_error(y_valid, y_fipe_valid, squared=False)

df_metricas = df_metricas.append({'Modelo': 'Tabela Fipe',
                                  'RMSE': rmse, 'MAPE': mape, 'RMSLE': rmsle}, ignore_index=True)

df_metricas

,Modelo,RMSE,MAPE,RMSLE
0,Tabela Fipe,11388.497909,0.081253,0.145193


# Encode das variáveis categóricas

As variáveis categóricas e binárias devem passar por um processo de encode adequado antes de serem oferecidas ao modelo de Machine Learning. Para as variáveis multicategóricas nominais, será aplicado o One Hot Encoder. Para as variáveis multicategóricas ordinais, será utilizado o One Hot Encoder. Por fim, para as variáveis com duas categorias e binárias, os valores serão convertidos de forma manual.

## One Hot Encoder

In [16]:
var_ohe = ['Marca', 'Tipo', 'UF', 'Combustivel', 'Cambio', 'Direcao', 'Cor', 'Modelo']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
df_ohe = pd.DataFrame(ohe.fit_transform(X_train[var_ohe]).toarray(), columns=ohe.get_feature_names_out(), index=X_train.index)

X_train = X_train.join(df_ohe)
X_train = X_train.drop(var_ohe, axis=1)

X_train.head()

,Ano,Km,Potencia,FinalDePlaca,Portas,UnicoDono,AceitaTrocas,ArCondicionado,AirBag,Alarme,...,Modelo_X3,Modelo_X4,Modelo_X5,Modelo_X6,Modelo_X60,Modelo_XC,Modelo_XF,Modelo_XSARA,Modelo_YARIS,Modelo_ZAFIRA
695,2015,77200,1.0,9,4 portas,Não,Sim,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5093,2012,266000,3.0 - 3.9,3,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501,2019,95000,2.0 - 2.9,8,4 portas,Sim,Sim,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4077,2013,259000,2.0 - 2.9,7,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5089,2010,149791,2.0 - 2.9,0,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df_ohe = pd.DataFrame(ohe.transform(X_valid[var_ohe]).toarray(), columns=ohe.get_feature_names_out(), index=X_valid.index)

X_valid = X_valid.join(df_ohe)
X_valid = X_valid.drop(var_ohe, axis=1)

df_ohe = pd.DataFrame(ohe.transform(X_test[var_ohe]).toarray(), columns=ohe.get_feature_names_out(), index=X_test.index)

X_test = X_test.join(df_ohe)
X_test = X_test.drop(var_ohe, axis=1)

## Ordinal Encoder

In [18]:
var_ode = ['Potencia']

from sklearn.preprocessing import OrdinalEncoder

ode = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[var_ode] = ode.fit_transform(X_train[var_ode])

X_train.head()

,Ano,Km,Potencia,FinalDePlaca,Portas,UnicoDono,AceitaTrocas,ArCondicionado,AirBag,Alarme,...,Modelo_X3,Modelo_X4,Modelo_X5,Modelo_X6,Modelo_X60,Modelo_XC,Modelo_XF,Modelo_XSARA,Modelo_YARIS,Modelo_ZAFIRA
695,2015,77200,0.0,9,4 portas,Não,Sim,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5093,2012,266000,10.0,3,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501,2019,95000,9.0,8,4 portas,Sim,Sim,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4077,2013,259000,9.0,7,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5089,2010,149791,9.0,0,4 portas,Não,Sim,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X_valid[var_ode] = ode.transform(X_valid[var_ode])

X_test[var_ode] = ode.transform(X_test[var_ode])

## Encoder Manual

### Features booleanas

In [20]:
var_bool = df.select_dtypes('bool').columns

X_train[var_bool] = X_train[var_bool].astype('int64')
X_valid[var_bool] = X_valid[var_bool].astype('int64')
X_test[var_bool] = X_test[var_bool].astype('int64')

X_train[var_bool].head()

,ArCondicionado,AirBag,Alarme,TravaEletrica,VidroEletrico,Som,SensorDeRe,CameraDeRe,Blindado
695,1,0,0,0,0,0,0,0,0
5093,1,1,1,1,1,0,1,0,0
501,0,0,0,0,0,0,0,0,0
4077,1,1,1,1,1,1,0,1,0
5089,1,1,1,1,1,1,1,0,0


### Features com duas categorias

In [21]:
X_train['Portas'] = X_train['Portas'].replace({'4 portas': 4, '2 portas': 2})
X_valid['Portas'] = X_valid['Portas'].replace({'4 portas': 4, '2 portas': 2})
X_test['Portas'] = X_test['Portas'].replace({'4 portas': 4, '2 portas': 2})

In [22]:
X_train['UnicoDono'] = X_train['UnicoDono'].replace({'Sim': 1, 'Não': 0})
X_valid['UnicoDono'] = X_valid['UnicoDono'].replace({'Sim': 1, 'Não': 0})
X_test['UnicoDono'] = X_test['UnicoDono'].replace({'Sim': 1, 'Não': 0})

In [23]:
X_train['AceitaTrocas'] = X_train['AceitaTrocas'].replace({'Sim': 1, 'Não': 0})
X_valid['AceitaTrocas'] = X_valid['AceitaTrocas'].replace({'Sim': 1, 'Não': 0})
X_test['AceitaTrocas'] = X_test['AceitaTrocas'].replace({'Sim': 1, 'Não': 0})

In [24]:
X_train.head()

,Ano,Km,Potencia,FinalDePlaca,Portas,UnicoDono,AceitaTrocas,ArCondicionado,AirBag,Alarme,...,Modelo_X3,Modelo_X4,Modelo_X5,Modelo_X6,Modelo_X60,Modelo_XC,Modelo_XF,Modelo_XSARA,Modelo_YARIS,Modelo_ZAFIRA
695,2015,77200,0.0,9,4,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5093,2012,266000,10.0,3,4,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501,2019,95000,9.0,8,4,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4077,2013,259000,9.0,7,4,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5089,2010,149791,9.0,0,4,0,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modelos com alto poder preditivo

In [25]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [26]:
from sklearn.metrics import mean_squared_error

y_pred = dtr.predict(X_valid)

rmse  = mean_squared_error(y_valid, y_pred, squared=False)
mape  = mean_absolute_percentage_error(y_valid, y_pred)
rmsle = mean_squared_log_error(y_valid, y_pred, squared=False)

df_metricas = df_metricas.append({'Modelo': 'DecisionTreeRegressor',
                                  'RMSE': rmse, 'MAPE': mape, 'RMSLE': rmsle}, ignore_index=True)

df_metricas

,Modelo,RMSE,MAPE,RMSLE
0,Tabela Fipe,11388.497909,0.081253,0.145193
1,DecisionTreeRegressor,32264.058618,0.181102,0.338476


In [27]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(random_state=42)
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [28]:
y_pred = rfr.predict(X_valid)

rmse  = mean_squared_error(y_valid, y_pred, squared=False)
mape  = mean_absolute_percentage_error(y_valid, y_pred)
rmsle = mean_squared_log_error(y_valid, y_pred, squared=False)

df_metricas = df_metricas.append({'Modelo': 'RandomForestRegressor',
                                  'RMSE': rmse, 'MAPE': mape, 'RMSLE': rmsle}, ignore_index=True)

df_metricas

,Modelo,RMSE,MAPE,RMSLE
0,Tabela Fipe,11388.497909,0.081253,0.145193
1,DecisionTreeRegressor,32264.058618,0.181102,0.338476
2,RandomForestRegressor,24738.704276,0.140376,0.202615


In [29]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [30]:
y_pred = xgb.predict(X_valid)

rmse  = mean_squared_error(y_valid, y_pred, squared=False)
mape  = mean_absolute_percentage_error(y_valid, y_pred)
rmsle = mean_squared_log_error(y_valid, y_pred, squared=False)

df_metricas = df_metricas.append({'Modelo': 'XGBRegressor',
                                  'RMSE': rmse, 'MAPE': mape, 'RMSLE': rmsle}, ignore_index=True)

df_metricas

,Modelo,RMSE,MAPE,RMSLE
0,Tabela Fipe,11388.497909,0.081253,0.145193
1,DecisionTreeRegressor,32264.058618,0.181102,0.338476
2,RandomForestRegressor,24738.704276,0.140376,0.202615
3,XGBRegressor,25694.715061,0.136843,0.201183


# Se não pode vencê-los, junte-se a eles

Na próxima etapa, o valor de preço da Tabela Fipe será incorporado ao modelo como uma variável independente, visando bater o resultado apresentado pelo baseline.